In [1]:
import pandas as pd

In [2]:
# Dict of files and tabs to scan through to extract applicable data
pres_election_sheets = {
    "federalelections2004.xlsx": "Table 2. Pres Elec & Pop Vote",
    "federalelections2008.xlsx": "Table 2. Electoral &  Pop Vote",
    "federalelections2012.xlsx": "Table 2. Electoral &  Pop Vote",
    "federalelections2016.xlsx": "Table 2. Electoral &  Pop Vote",
    "federalelections2020.xlsx": "3. Table 2 Electoral & Pop Vote",
}


state_turnout_sheets = {
    "federalelections2004.xlsx": "Table 3. GE Votes by State",
    "federalelections2008.xlsx": "Table 3.GEVotes for Pres, H & S",
    "federalelections2012.xlsx": "Table 3. GEVotes for Pres, H, S",
    "federalelections2016.xlsx": "Table 3. GEVotes for Pres, H, S",
    "federalelections2020.xlsx": "4. Table 3 GEVotefor Pres, H, S",
}

# %%
pres_df = pd.DataFrame()

In [3]:
# Cycle through each file in the dict and clean up imported file to get rectangular data
for key, value in pres_election_sheets.items():

    tmp = pd.read_excel(io=key, sheet_name=value, dtype=str)

    # get election year from first column
    tmp_year = tmp.columns[0][:4]

    # dynamically find top of table (varies by year)
    while True:
        # if the top left cell value is 'STATE' then it is highest potential starting point of a table
        if "state" in str(tmp.iloc[0, 0]).lower():
            tmp.columns = list(["state"] + list(tmp.iloc[0, :][1:]))
            tmp = tmp.iloc[1:,]
            tmp.columns = [
                str(i).lower() for i in tmp.columns
            ]  # convert nan's to strings and all lowercase
            new_columns = list(tmp.columns)
            for i in range(len(new_columns)):
                if (new_columns[i] == "nan") & (i == 0):
                    new_columns[i] == "state"  # assume first column is state
                elif (new_columns[i] == "nan") & (i == len(new_columns)):
                    new_columns[i] == "total"  # assume last column is total
                elif new_columns[i] == "nan":
                    new_columns[i] = new_columns[i - 1]

            if str(tmp.iloc[0, 0]) == "AL":
                tmp.columns = new_columns  # reset the column names
            elif (str(tmp.iloc[0, 0]) == "nan") & (str(tmp.iloc[1, 0]) == "AL"):
                first_row = [str(i).lower() for i in tmp.iloc[0, :]]
                for i in range(len(new_columns)):
                    if first_row[i] == "nan":
                        next  # no change to new_columns[j]
                    else:
                        new_columns[i] = new_columns[i] + " " + first_row[i]
                tmp.columns = new_columns  # reset the modified columns
                tmp = tmp.iloc[1:,]
            elif (str(tmp.iloc[1, 0]) == "nan") & (str(tmp.iloc[2, 0]) == "AL"):
                first_row = [str(i).lower() for i in tmp.iloc[1, :]]
                for i in range(len(new_columns)):
                    if first_row[i] == "nan":
                        next  # no change to new_columns[j]
                    else:
                        new_columns[i] = new_columns[i] + " " + first_row[i]
                tmp.columns = new_columns  # reset the modified columns
                tmp = tmp.iloc[2:,]

            tmp = tmp[
                [c for c in list(tmp.columns) if "electoral" not in c]
            ]  # exclude electoral college columns

            tmp = tmp.reset_index(drop=True)
            for row in tmp.itertuples():
                if "total" in str(row[1]).lower():
                    total_row = row.Index  # this is the bottom of the table
                    tmp = tmp.iloc[: total_row + 1,]

            tmp["year"] = tmp_year

            final_columns = list(tmp.columns)
            final_columns = [str(i).replace("popular vote ", "") for i in final_columns]
            final_columns = [str(i).replace(" vote", "") for i in final_columns]
            tmp.columns = final_columns

            tmp = tmp.melt(
                id_vars=["state", "year"], var_name="candidate", value_name="votes"
            )

            tmp["votes"] = tmp["votes"].astype(int)

            # assign party
            tmp["party"] = "third party"  # default
            tmp.loc[tmp["candidate"].str.contains(r"total"), "party"] = "total"
            tmp.loc[tmp["candidate"].str.contains(r"\(r\)"), "party"] = "republican"
            tmp.loc[tmp["candidate"].str.contains(r"\(d\)"), "party"] = "democrat"

            # clean candidate
            tmp["candidate"] = tmp["candidate"].str.replace(r" (d)", "")
            tmp["candidate"] = tmp["candidate"].str.replace(r" (r)", "")
            tmp["candidate"] = tmp["candidate"].str.replace(
                r"trunp", "trump"
            )  # This is a typo that I fixed already in the excel file

            # clean state with asterick's
            tmp["state"] = tmp["state"].str.replace(r"*", "")

            break
        else:
            tmp = tmp.iloc[1:,]

    pres_df = pd.concat([pres_df, tmp], axis=0)
    del key, value, i, row, tmp, tmp_year, new_columns, total_row

In [4]:
# Clean up column names and formatting
pres_df = pres_df[["state", "year", "party", "candidate", "votes"]]
pres_df = pres_df.rename(
    columns={
        "state": "State",
        "year": "Year",
        "party": "Party",
        "candidate": "Candidate",
        "votes": "Votes",
    }
)
pres_df["Party"] = pres_df["Party"].str.title()
pres_df["Candidate"] = pres_df["Candidate"].str.title()
pres_df["State"] = pres_df["State"].str.strip()
pres_df["State"] = pres_df["State"].str.replace(":", "")

In [5]:
# Import the 2024 data, which comes from a separate file because it was manually compiled
pres_df_2024 = pd.read_csv(
    "federalelections2024_manual.csv",
    dtype={"State": str, "Year": str, "Party": str, "Candidate": str, "Votes": int},
)

pres_df_2024 = pd.concat(
    [
        pres_df_2024,
        (
            pres_df_2024.filter(["State", "Year", "Votes"])
            .groupby(by=["State", "Year"], as_index=False)
            .agg({"Votes": "sum"})
            .assign(Party="Total", Candidate="Total")
            .reindex(["State", "Year", "Party", "Candidate", "Votes"], axis=1)
        ),
    ]
)

pres_df_2024 = pd.concat(
    [
        pres_df_2024,
        (
            pres_df_2024.groupby(["Year", "Party", "Candidate"], as_index=False)
            .agg({"Votes": "sum"})
            .assign(State="Total")
            .reindex(["State", "Year", "Party", "Candidate", "Votes"], axis=1)
        ),
    ]
)

# Combine data frames
pres_df = pd.concat([pres_df, pres_df_2024], axis=0)
del pres_df_2024

In [6]:
# Order categorical columns
pres_df.Party = pd.Categorical(pres_df.Party, categories=["Republican", "Democrat", "Third Party", "Total"])
pres_df.State = pd.Categorical(pres_df.State, categories=['AK','AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'Total'])

pres_df.sort_values(["Party","State"], inplace=True)

In [7]:
# Write file
pres_df.to_csv("compiled_fed_elections_2004-2024.csv", index=False)